# pega-pronunciamentos-html

 Esse script usa os csvs gerados em `pega-links` e extrai os pronunciamentos para os quais existem links para conteúdo em html.

#### TO DO:
- Implementar função `download_pdf()`

#### Importa pacotes

In [49]:
from bs4 import BeautifulSoup
import pandas as pd
import pprint as pp
import os
import re
import requests
import time
from tqdm._tqdm_notebook import tqdm_notebook as tqdm


import os.path
from os import path
from datetime import datetime

In [3]:
# Chama tqdm para monitorar progresso no pandas
tqdm.pandas()

C:\DataScience\ProgramData\Anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


#### Lê dados externos

In [4]:
def read_data(name, database):
    
    '''
    Lê arquivos .csv usando o pandas. Os arquivos devem
    seguir o padrão de nome '{nome-parlamentar}-{database}-metadada.csv'
    
    Parâmetros:
    
    name -> Nome do parlamentar separado por hífen (como "jair-bolsonaro")
    database -> "plenario" ou "comissao"
    '''
    
    fpath = "../data/tables/{name}-{database}-metadata.csv".format(
        name = name, 
        database = database
    )
    
    df = pd.read_csv(fpath, dtype='str')
    
    return df

#### Funções para extrair dados de um link e salvá-lo

In [5]:
def make_request(url):
    
    '''
    Recebe uma url, faz requisição get
    e retorna uma string de texto.
    '''

    headers = {"User-Agent":"Rodrigo Menegat, jornalista (rodrigoschuinski@gmail.com)"}
    r = requests.get(url, headers = headers)
    r.encoding = 'utf-8'
    return r.text

In [6]:
def make_soup(data):
    
    '''
    Recebe uma string de texto e retorna um objeto do BeautifulSoup.
    '''
    
    # Preciso retirar as tags </b> do texto porque
    # o desenvolvedor da Câmara colocou elas fora
    # de ordem e isso faz o raspador enlouquecer.
    #                 ¯\_(ツ)_/¯
    data = data.replace("<b>","").replace("</b>","")
    soup = BeautifulSoup(data, 'html.parser')
    return soup

In [7]:
def scrape_content_plen(soup):
    
    '''
    Usa o BeautifulSoup para extrair o conteúdo da URL
    em que está o discurso. Retorna uma string de texto.
    '''
        
    paragraph = soup.find( "p", attrs = {"align":"justify"} )
    
    if not isinstance(paragraph, type(None)):


        # Esse passo a mais é necessário porque, em alguns casos, há mais de uma tag 'font' dentro do 'p'.
        # Nessas ocasiões, não era retornado todo o texto, mas apenas o conteúdo da primeira tag
        fonts = paragraph.find_all("font")

        text = [ font.text.strip() for font in fonts ]
        text = ' '.join(text)


        return text

In [8]:
def scrape_content_com(soup):
    
    '''
    Usa o BeautifulSoup para extrair o conteúdo da URL
    em que está o discurso. Retorna uma string de texto.
    '''
    
    fonts = soup.find_all("font")
    fonts = [ font.text.strip() for font in fonts ]
    content = ' '.join(fonts)
    
    return content

In [9]:
def save_txt(content, path, *args):
    '''
    Salva um arquivo txt no path designado.
    O array args é usado para gerar o nome do arquivo.
    '''
    
    file_id = '-'.join(args)
    file_id = file_id.replace("/","").replace(".","-")
    fname = path + file_id + ".txt"
    
    print(fname)
   
    with open(fname, 'w+') as file:
        file.write(content)
        
    return

In [10]:
def download_pdf(url, date, path):
    
    '''
    Função para baixar o pdf do documento
    onde está registrado um pronunciamento.
    '''
    
    r = requests.get(url)
    
    # TO DO - implementar captura de sessões conjuntas,
    # cujo link leva para um PDF viewer feioso
    
    # Pega redirecionamentos
    if r.history:    
        # Acessa a tag de redirecionamento e pega o contúdo
        r = requests.get(url)
        soup = make_soup(r.text)
        
       # Acha o link de redirecionamento
        meta = soup.find('meta',attrs={'http-equiv':'Refresh'})
        meta = str(meta)
        meta = re.search(".*URL=(.*)\" http.*", meta)
        
        # Se encontrado, refaz a solicitação
        if meta:
            url = meta.group(1)
            r =  requests.get(url)
            
            # Seleciona página da declaração para salvar no nome do arquivo
            page_no = re.search("page=(\d+)", url).group(1)
            fpath = "{path}{date}-pg-{page_no}.pdf"
            fpath = fpath.format(path = path, 
                                 date = date, 
                                 page_no = page_no)
    
            if not os.path.isfile(fpath):
                with open(fpath, 'wb') as f:
                    f.write(r.content)
    
    else:
        print("Couldn't download the following url:\n" + url)
    
    return

#### Função que, via df.apply, as funções acima em cada linha do dataframe

In [53]:
def scrape_row_plen(row):
    
    '''
    Função para executar via df.apply()
    que acessa os dados de cada linha do
    dataframe e executa as operações ne-
    cessárias para acessar os dados. Usar
    apenas com a tabela com discursos feitos
    no plenário.
    '''
    
    if row.Discurso != '-':
    
        url = row.Discurso
        data = make_request(url)
        soup = make_soup(data)
        txt = scrape_content_plen(soup)
        
        if not isinstance(txt, type(None)):
            #print(type(txt))
            ano  = str(datetime.strptime(row.Data,'%d/%M/%Y').year)
            dir_path = "../data/txts/plenario/{}/".format(ano)
            
            if not path.isdir(dir_path):
                os.mkdir(dir_path)                    
                    
            file_id = [ str(row.name), row.Data, row.Hora, str(row.Fase), row.Sessão ]
            save_txt(txt, dir_path, *file_id)

            time.sleep(.2)
    
    else:
        url = row.Publicação
        dir_path = "../data/pdfs/plenario/"
        date = row.Data.replace('/','-')
        # download_pdf(url, date, path)

        return
    

In [12]:
def scrape_row_com(row):

    '''
    Função para executar via df.apply()
    que acessa os dados de cada linha do
    dataframe e executa as operações ne-
    cessárias para acessar os dados. Usar
    apenas com a tabela com discursos feitos
    no plenário.
    '''
    
    url = row.Texto
    data = make_request(url)
    soup = make_soup(data)
    txt = scrape_content_com(soup)

    path = "../data/txts/comissao/"
    file_id = [row.Data, row.Hora, row.Reunião]
    save_txt(txt, path, *file_id)

    # time.sleep(.2)
    
    return

#### Roda funções

In [13]:
def scrape_df(df):
    
    '''
    Função que encapsula os métodos de raspagem aplicados via df.apply().
    Ela é útil para detectar com qual tabela (plenário ou comissões)
    estamos lidando e fazer a operação correta, sem precisar se preocupar
    com selecionar a função específica para o dataframe.
    '''

    if 'Sessão' in df.columns:
        df.progress_apply(scrape_row_plen, axis = 1)
    elif 'Comissão' in df.columns:
        df.progress_apply(scrape_row_com, axis = 1)
    else:
        raise Exception("O dataframe não está em um formato conhecido.")
        
    return

In [14]:
#df_plen = read_data('01-01-2019', 'plenario')

In [17]:
len(df_plen)

20390

In [ ]:
qtd = 2000
init = 0

for i in range(0,9):
    fim = i+qtd+init
    print('count', init, fim)
    scrape_df(df_plen[init:fim])
    init = fim

In [ ]:
datas = [("01-01-2001"),
         ("01-01-2003"),
         ("01-01-2005"),
         ("01-01-2007"),
         ("01-01-2019")]

dados = pd.DataFrame()
for d in datas:
    print('Capturando: {}'.format(d))
    dados = read_data(d, 'plenario')#pd.concat([dados,read_data(d, 'plenario')])
    scrape_df(dados)

Capturando: 01-01-2001


../data/txts/plenario/2001/0-28122001-9h26-PEQUENO EXPEDIENTE-007-5-51-E.txt
../data/txts/plenario/2001/1-28122001-9h20-PEQUENO EXPEDIENTE-007-5-51-E.txt
../data/txts/plenario/2001/2-28122001-9h14-PEQUENO EXPEDIENTE-007-5-51-E.txt
../data/txts/plenario/2001/3-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/4-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/5-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/6-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/7-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/8-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/9-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/10-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/11-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/12-27122001-22h46-ORDEM DO DIA-004-5-51-C.txt
../data/txts/plenario/2001/13-27122001-22h46-O

../data/txts/plenario/2001/109-27122001-13h28-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/110-27122001-13h24-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/111-27122001-13h22-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/112-27122001-13h22-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/113-27122001-13h22-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/114-27122001-13h16-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/115-27122001-13h10-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/116-27122001-13h04-PEQUENO EXPEDIENTE-006-5-51-E.txt
../data/txts/plenario/2001/117-26122001-15h52-COMUNICAÇÕES PARLAMENTARES-005-5-51-E.txt
../data/txts/plenario/2001/118-26122001-15h50-COMUNICAÇÕES PARLAMENTARES-005-5-51-E.txt
../data/txts/plenario/2001/119-26122001-15h48-COMUNICAÇÕES PARLAMENTARES-005-5-51-E.txt
../data/txts/plenario/2001/120-26122001-15h48-COMUNICAÇÕES PARLAMENTARES-005-5-51-E.txt
../data/

../data/txts/plenario/2001/208-19122001-16h56-COMUNICAÇÕES PARLAMENTARES-002-5-51-E.txt
../data/txts/plenario/2001/209-19122001-16h54-COMUNICAÇÕES PARLAMENTARES-002-5-51-E.txt
../data/txts/plenario/2001/210-19122001-16h52-COMUNICAÇÕES PARLAMENTARES-002-5-51-E.txt
../data/txts/plenario/2001/211-19122001-16h52-COMUNICAÇÕES PARLAMENTARES-002-5-51-E.txt
../data/txts/plenario/2001/212-19122001-16h50-COMUNICAÇÕES PARLAMENTARES-002-5-51-E.txt
../data/txts/plenario/2001/213-19122001-16h40-COMUNICAÇÕES PARLAMENTARES-002-5-51-E.txt
../data/txts/plenario/2001/214-19122001-16h38-ORDEM DO DIA-002-5-51-E.txt
../data/txts/plenario/2001/215-19122001-16h28-GRANDE EXPEDIENTE-002-5-51-E.txt
../data/txts/plenario/2001/216-19122001-16h26-GRANDE EXPEDIENTE-002-5-51-E.txt
../data/txts/plenario/2001/217-19122001-16h20-GRANDE EXPEDIENTE-002-5-51-E.txt
../data/txts/plenario/2001/218-19122001-16h08-GRANDE EXPEDIENTE-002-5-51-E.txt
../data/txts/plenario/2001/219-19122001-16h08-GRANDE EXPEDIENTE-002-5-51-E.txt
../

../data/txts/plenario/2001/309-18122001-16h14-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/310-18122001-16h08-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/311-18122001-16h02-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/312-18122001-16h00-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/313-18122001-16h00-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/314-18122001-16h00-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/315-18122001-15h58-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/316-18122001-15h46-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/317-18122001-15h44-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/318-18122001-15h38-COMUNICAÇÕES PARLAMENTARES-001-5-51-E.txt
../data/txts/plenario/2001/319-18122001-15h30-ORDEM DO DIA-001-5-51-E.txt
../data/txts/plenario/2001/320-18122001-15h30-

...

../data/txts/plenario/2001/6521-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6522-19092001-17h12-BREVES COMUNICAÇÕES-175-3-51-O.txt
../data/txts/plenario/2001/6523-19092001-17h12-BREVES COMUNICAÇÕES-175-3-51-O.txt
../data/txts/plenario/2001/6524-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6525-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6526-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6527-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6528-19092001-17h12-BREVES COMUNICAÇÕES-175-3-51-O.txt
../data/txts/plenario/2001/6529-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6530-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6531-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6532-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../data/txts/plenario/2001/6533-19092001-17h12-ORDEM DO DIA-175-3-51-O.txt
../d

../data/txts/plenario/2001/6624-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6625-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6626-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6627-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6628-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6629-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6630-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6631-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6632-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6633-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6634-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6635-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6636-19092001-12h46-ORDEM DO DIA-17-3-51-N.txt
../data/txts/plenario/2001/6637-190920

../data/txts/plenario/2001/6733-18092001-16h48-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6734-18092001-16h42-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6735-18092001-16h38-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6736-18092001-16h30-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6737-18092001-16h24-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6738-18092001-16h18-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6739-18092001-16h18-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6740-18092001-16h16-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6741-18092001-16h16-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6742-18092001-16h16-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6743-18092001-16h10-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6744-18092001-16h02-GRANDE EXPEDIENTE-173-3-51-O.txt
../data/txts/plenario/2001/6745-18092001

../data/txts/plenario/2001/6837-17092001-13h04-PEQUENO EXPEDIENTE-171-3-51-O.txt
../data/txts/plenario/2001/6838-14092001-12h46-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6839-14092001-12h34-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6840-14092001-12h24-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6841-14092001-12h14-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6842-14092001-12h14-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6843-14092001-12h00-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6844-14092001-11h50-HOMENAGEM-170-3-51-O.txt
../data/txts/plenario/2001/6845-14092001-11h34-COMUNICAÇÕES PARLAMENTARES-170-3-51-O.txt
../data/txts/plenario/2001/6846-14092001-11h26-COMUNICAÇÕES PARLAMENTARES-170-3-51-O.txt
../data/txts/plenario/2001/6847-14092001-11h16-COMUNICAÇÕES PARLAMENTARES-170-3-51-O.txt
../data/txts/plenario/2001/6848-14092001-10h50-GRANDE EXPEDIENTE-170-3-51-O.txt
../data/txts/plenario/2001/6849-14092001-10h34-GRANDE EXPEDIENTE-170

../data/txts/plenario/2001/6945-13092001-11h06-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6946-13092001-11h00-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6947-13092001-10h54-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6948-13092001-10h54-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6949-13092001-10h52-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6950-13092001-10h52-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6951-13092001-10h52-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6952-13092001-10h46-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6953-13092001-10h42-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6954-13092001-10h40-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6955-13092001-10h38-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/plenario/2001/6956-13092001-10h38-BREVES COMUNICAÇÕES-168-3-51-O.txt
../data/txts/ple

In [ ]:
#scrape_df(dados)

In [38]:
for x,i in dados.iterrows():
    print(datetime.strptime(i['Data'],'%d/%M/%Y').year)#loc[0,'Data']
    break

2010


In [16]:
# https://medium.com/@sergioprates/intelig%C3%AAncia-artificial-an%C3%A1lise-de-sentimento-sobre-as-avalia%C3%A7%C3%B5es-do-imdb-parte-1-5d1fd27afc80
# https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk
# https://medium.com/@alegeorgelustosa/an%C3%A1lise-de-sentimentos-em-python-2a7d04a836e0
# https://github.com/rdenadai/sentiment-analysis-2018-president-election
# 
import datetime

aa = a.Orador.apply(lambda x: x.split(","))
a["Ano"]     =  a.Data.apply(lambda x: datetime.datetime.strptime(x , "%d/%m/%Y" ).year)
a["Dep."]    =  [ d[0] for d in aa]
a["Partido"] =  [ d[1] if len(d) > 1 else 'ND' for d in aa ]


In [17]:
a.head()

,Data,Fase,Hora,Orador,Ano,Dep.,Partido
0,20/12/2018,GRANDE EXPEDIENTE,15h28,"GIOVANI FELTES, MDB-RS",2018,GIOVANI FELTES,MDB-RS
1,20/12/2018,GRANDE EXPEDIENTE,15h20,"BEBETO, PSB-BA",2018,BEBETO,PSB-BA
2,20/12/2018,GRANDE EXPEDIENTE,15h00,"ALFREDO KAEFER, PP-PR",2018,ALFREDO KAEFER,PP-PR
3,20/12/2018,PEQUENO EXPEDIENTE,14h56,"MARCONDES GADELHA, PSC-PB",2018,MARCONDES GADELHA,PSC-PB
4,20/12/2018,PEQUENO EXPEDIENTE,14h48,"ROBERTO DE LUCENA, PODE-SP",2018,ROBERTO DE LUCENA,PODE-SP


In [ ]:
a = pd.DataFrame(columns=['b','d'])
x = [ 'GIOVANI FELTES, MDB-RS','DIAS TOFFOLI (PRESIDENTE DO SUPREMO TRIBUNAL FEDERAL)']

for i in x:
    i = i.split(",")
    print(i, len(i))

In [ ]:
#print(df_plen[1024:1026])

In [ ]:
#df_com = read_data('jair-bolsonaro', 'comissao')

In [ ]:
#scrape_df(df_com)